In [1]:
import numpy as np
x=np.random.rand(10,1,28,28)
print(x.shape)
print(x[0].shape)
print(x[1].shape)
print(x[2].shape)
print(x[3].shape)
print(x[0][0])

(10, 1, 28, 28)
(1, 28, 28)
(1, 28, 28)
(1, 28, 28)
(1, 28, 28)
[[7.60989144e-01 5.02146845e-01 7.35748192e-01 1.74289921e-01
  5.31703281e-01 6.60483490e-01 5.11754926e-01 5.42569790e-01
  3.57973805e-01 9.14214436e-02 4.52842821e-01 2.62657476e-01
  2.74072927e-01 4.44453069e-01 8.80709282e-01 2.30317242e-01
  8.02069924e-01 4.89342699e-01 7.49276885e-01 1.41115172e-01
  4.49884366e-02 6.21382665e-01 6.88237838e-01 1.76043027e-01
  2.25960272e-01 7.25139954e-01 6.74306314e-03 5.21314317e-01]
 [4.51657790e-01 8.45072852e-01 7.55138011e-01 7.42021264e-02
  6.13673620e-01 4.70760568e-01 8.85490991e-01 9.60190101e-02
  6.02271817e-01 6.41706700e-01 1.48738051e-01 3.62898568e-01
  1.61177059e-01 3.18496647e-02 1.63450103e-01 9.64850032e-01
  6.47219979e-01 9.71038194e-01 4.76668933e-01 8.45642545e-01
  5.08152159e-01 7.73225033e-01 2.18118449e-01 4.62227779e-01
  5.55341820e-02 8.94230801e-01 8.99745532e-01 6.81389085e-01]
 [4.75863161e-01 3.12082765e-01 4.57380356e-01 3.44785115e-01
  6.

In [3]:
import sys, os
sys.path.append(os.pardir)
import matplotlib.pyplot as plt
import pandas as pd
from common.util import im2col

In [5]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride=1,pad=0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2,5,5,stride=1,pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [ ]:
class Convoution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W=W
        self.b=b
        self.stride = stride
        self.pad=pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1+(H+2*self.pad-FH)/self.stride)
        out_w = int(1+(W+2*self.pad-FW)/self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        
        return out

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

In [ ]:
class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28), 
                 conv_param={'filter_num': 30, 'filter_size':5, 'pad':0, 'stride':1}, 
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad)/filter_strdie + 1
        pool_output_size = int(filter_num*(conv_output_size/2)*(conv_output_size/2))

        # 매개변수 초기화
        self.params={}
        self.params['W1'] = weight_init_std*np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layer.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads={}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db